In [1]:
def decorator(func):
    print(func)
    return "test"

@decorator
def f():
    pass

print(f)
print("----")

def decorator(func):
    return func

@decorator
def f():
    pass

print(f)

<function f at 0x000001F3B1EF3C80>
test
----
<function f at 0x000001F3B1EF39D8>


In [2]:
def thingy(func):
    print("Created : ", func)
    return func

@thingy
def f():
    print("Do something")

print(f())
print("----")
f()

Created :  <function f at 0x000001F3B1EF3730>
Do something
None
----
Do something


In [3]:
def woggle(func):
    print("Used decorator on:", func)
    def wrapper(*args, **kws):
        print("Called:", func)
        return func(*args, **kws)
    return wrapper

@woggle
def g():
    pass

@woggle
def f():
    print("Do something")
    return "Thingy"

print("----")
print(f)
print("----")
print(f())
print("----")
print(g)
print("----")
print(g())

Used decorator on: <function g at 0x000001F3B1F04268>
Used decorator on: <function f at 0x000001F3B1F04A60>
----
<function woggle.<locals>.wrapper at 0x000001F3B1F04D08>
----
Called: <function f at 0x000001F3B1F04A60>
Do something
Thingy
----
<function woggle.<locals>.wrapper at 0x000001F3B1F049D8>
----
Called: <function g at 0x000001F3B1F04268>
None


In [13]:
import functools

def woggle(func):
    @functools.wraps(func)
    def wrapper(*args, **kws):
        print("Called:", func)
        return func(*args, **kws)
    return wrapper

@woggle
def f():
    """
    docs
    """
    print("Do something")
    return "Thingy"

print(f, f.__doc__)
print("----")
print(f())

<function f at 0x000001F3B1F36D90> 
    docs
    
----
Called: <function f at 0x000001F3B1F36598>
Do something
Thingy


In [5]:
import functools

def log(func):
    @functools.wraps(func)
    def wrapper(*args, **kws):
        ... # Elided
    #wrapper = functools.wraps(func)(wrapper)
    return wrapper

@log
def test(name, value):
    "doc string"
    if value:
        print(name, value)
        return name
    else:
        return ""
#test = log(test)

print(test)
print(test.__doc__)

print(test("first", 1))

print(test("zero", 0))

print(test.__name__)
print(test)

<function test at 0x000001F3B1EF39D8>
doc string
None
None
test
<function test at 0x000001F3B1EF39D8>


In [6]:
def log(prefix):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kws):
            ...
            return func(*args, **kws)
        return wrapper
    return decorator

#adecorator = log("Test function")
#awrapper = adecorator(test)
#test = awrapper

@log("Test function")
def test(name, value):
    """Doc string"""
    if value:
        print(name, value)
        return name
    else:
        return ""
#test = log("Test function")(test)

print(test, test.__doc__)

print(test("first", 1))
print(test("zero", 0))

<function test at 0x000001F3B1F042F0> Doc string
first 1
first



#### Magic Methods
#### __init__: Constructor
#### __mul__ : Multiplication as a * b (a.__mul__(b))
#### __matmul__ : Matrix multiplication: a @ b
#### Many others

In [7]:
class Vector(object):
    def __init__(self, data):
        self.data = list(data)

    def __mul__(self, rhs): # self * rhs
        if hasattr(rhs, "data"):
            return Vector(x * y for x, y in zip(self.data, rhs.data))
        else:
            return Vector(x * rhs for x in self.data)

    # Call for lhs * self and lhs does not have __mul__ or doesn't support Vectors.
    def __rmul__(self, lhs): # lhs * self
        return self * lhs

    def __imul__(self, rhs):
        for i in range(len(self.data)):
            self.data[i] *= rhs
        return self

    def __str__(self): # str(o)
        return "<{}>".format(self.data)

    def __repr__(self): # repr(o)
        return "Vector({})".format(repr(self.data))

    def __eq__(self, other):
        return self.data == other.data

In [8]:
v = Vector(range(10))
print(v)
print(v * v)

print(v * 2)

print(2 * 3)
print((2).__mul__(3))

print(2 * v) # (2).__mul__(v)

print(v, str(v), repr(v))


w = Vector(range(10))
print(v == w)

<[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]>
<[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]>
<[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]>
6
6
<[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]>
<[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]> <[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]> Vector([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
True


In [9]:
class TypedList(object):
    def __init__(self, tpe):
        self.tpe = tpe
        self.data = []

    def append(self, v):
        if not isinstance(v, self.tpe):
            raise ValueError(v)
        self.data.append(v)

    def __iter__(self):
        return iter(self.data)

    def __getitem__(self, ind):
        return self.data[ind]

    def __setitem__(self, ind, val): # self[ind] = val
        if not isinstance(val, self.tpe):
            raise ValueError(val)
        self.data[ind] = val

lst = TypedList(int)
lst.append(2)

for x in lst:
    print(x)

print(lst[0])

lst[0] = 5

print(lst[0])

2
2
5


In [10]:
class FakeFunction(object):
    def __call__(self, x, y):
        return (x,y)

    def backwards(self, x, y):
        ...

f = FakeFunction()

print(f(1,2))

(1, 2)


In [11]:
class ContextMan(object):
    def __enter__(self):
        return 42

    def __exit__(self, *exc):
        print(exc)

cm = ContextMan()
with cm as x:
    print(x)

42
(None, None, None)
